# "Visualize, describe, compare"- nanoinformatics approaches for material-omics [Nanoparticle Atlas]

In [ ]:
from npAtlas_funcs import * #Import all functions from my NP Atlas

## Import base dataset
All 16 nanoparticle formulations plotted together and individually.

In [ ]:
# Init variables
path = 'C:/Users/20210014/OneDrive - TU Eindhoven/testingData/_nanoparticleAtlas/nanoparticleAtlas_matchedSettings' # Pay attention to the direction of the slash "/"
subFraction = 1 # Fraction to subsample the dataset for the MST (it's a computationally heavy operation and sometimes it cannot be computed for the 100% of the data in a normal PC)
labelHeader = 'sample' # Header name of the label column

In [ ]:
## Import dataset:
data = importData(path)
data

In [ ]:
## Clean Dataset
unfilteredDataset = npAtlasDataCleaning(data)
unfilteredDataset

In [ ]:
## Filter dataset ##

# OP1: targetCounts     --> drops samples where #Targets > #localizations
# OP2: fittingQuality   --> drops samples where rSquared < 0.8

option = 'fittingQuality'
filteredDataset = filterDataset(option, unfilteredDataset)
filteredDataset # Show dataset

In [ ]:
## Merge the samples that were done by different people (filteredDataset) and separate them for the reproducibility study (reprod_peopleDataset)
filteredReprod_peopleDataset, filteredDataset = separateReproducibilityExperiments(filteredDataset, labelHeader)

In [ ]:
##### COLOR PALETTE FOR NANOPARTICLE ATLAS ####
colorDict = {'100_medium':'#FFFA90', '100_high':'#FFF758',
            '200_low':'#FFE1C7', '200_medium':'#FFC390', '200_high':'#FFA558', 
            '300_low':'#C6F8FA', '300_medium':'#8DF1F6', '300_high':'#54EAF1', 
            '500_low': '#E3CAFB','500_medium': '#C894F7','500_high': '#AC5FF3',

            'NH2_pG0pM1': '#54AEAD','NH2_pG1pM3': '#BFE5A0','NH2_pG1pM1': '#FFFFBE', 'NH2_pG3pM1': '#FDBF6F','NH2_pG1pM0': '#E95C47'
            }

### Data exploration

In [ ]:
x = 'Cluster localizations'
y1 = 'channel 1 target count'
y2 = 'channel 2 target count'
title = 'Filtered dataset for samples where rSquared > 0.8'
filename = "qualityFilteredData"
scatterEDA(filteredDataset, x, y1, y2, title, colorDict, labelHeader, filename)

## Describe

In [ ]:
##### Calculate classic metrics #####
index, pdi, mcv, mean = metrics(filteredDataset, labelHeader)

In [ ]:
#Get the PDI value for the chosen formulation
formulation = '500_high'
pdiValue = pdi[formulation]
pdiFormulation = pd.DataFrame({'Feature': index, 'PDI': pdiValue})
print(f'The PDI values for the formulation {formulation} are {pdiFormulation}')

In [ ]:
## Simplify the dataset to plot most relevant features
smaller_dataset = reduceFeatures(filteredDataset)
smaller_dataset

In [ ]:
index = ['d', 'AR', 'Ab', 'Fab','Fc'] #Rename index if we don't want the automatic labelling
samples = uniqueNameSamples = list(set(smaller_dataset['sample'])) # Get all samples
compare = False # Flag to plot metrics between 2 samples
metricsPlot(smaller_dataset, index, mcv, samples, compare, labelHeader)

## Visualize
Clustering analysis:
1. PCA
2. t-SNE
3. UMAP
4. MST

In [ ]:
# Prepare the dataset for clustering algorithms: PCA, t-SNE AND UMAP
minmax = False # True for MST
scaledX, target, targetMass, uniqueLabels, features, dataMass = preprocess4clustering(filteredDataset, labelHeader, mean, minmax)

In [ ]:
# Prepare the dataset for clustering algorithms: MST
minmax = True # True for MST
subsampledDataset, subsampledTarget = subsampleData(filteredDataset, subFraction, labelHeader)
normalizedData, target_mst, _, _, _, _ = preprocess4clustering(subsampledDataset, labelHeader, mean, minmax)

In [ ]:
###
plt.subplots() # plot emtpy figure to set the style (idk why I get this bug)
sns.set_theme(style="darkgrid")
plt.close() # close empty figure
###

In [ ]:
plotLegend(colorDict) #Plot legend separately

### PCA

In [ ]:
nFeatures = 5 # How many top most important features do you want to see on the biplot?
pca2D(features, targetMass, scaledX, uniqueLabels, colorDict, nFeatures)

### t-SNE

In [ ]:
# Hypertune t-SNE
# perplexities = [50, 75, 100, 150, 300, 500, 550, 700, 1000] #we start with these and see if we have to adapt them
# tsneTuner(scaledX, targetMass, perplexities, colorDict)

In [ ]:
## Calculate t-SNE with the selected perplexity
perplexity = 1000
mytsne(scaledX, targetMass, perplexity, uniqueLabels, colorDict)

### UMAP

In [ ]:
# Hypertune UMAP
# n_neighbors = [10, 15, 25, 50, 75, 100, 200, 500, 1000] #we start with these and see if we have to adapt them
# umapTuner(scaledX, targetMass, n_neighbors, colorDict)

In [ ]:
n_neighbors = 15
samplePlots = True # Plot all samples separated by color
reducer, embedding_df = myumap(scaledX, targetMass, n_neighbors, uniqueLabels, colorDict, samplePlots)

#### Interactive UMAP

In [ ]:
mapper = reducer.fit(scaledX)

In [ ]:
hover_data = pd.DataFrame({'index':np.arange(len(dataMass)),
                           'Diameter':dataMass[:,0],
                           'Aspect ratio': dataMass[:,1],
                           'Total Ab': dataMass[:,2],
                           'Fab':dataMass[:,13],
                           'Fc': dataMass[:,22],
                           'Sample': targetMass})

In [ ]:
import umap.plot as umap_plot
umap_plot.output_notebook()

In [ ]:
p = umap_plot.interactive(mapper, labels=targetMass, hover_data=hover_data, point_size=2, background='#EAEAF2')
umap_plot.show(p)

#### FEATURE-COLORED UMAP

In [ ]:
featureToColor = ['Diameter', 'Aspect ratio', 'Cluster localizations', 'channel 1 target count', 'channel 2 target count', 'channel 1 true mean dark time', 'channel 2 true mean dark time', 'channel 1 mean bright time', 'channel 2 mean bright time']
featureColoredUmap(filteredDataset, featureToColor, embedding_df, 'umap_1', 'umap_2')

### MST

In [ ]:
filename = "complete"
mymst(normalizedData, target_mst, colorDict, subFraction, filename)

In [ ]:
## Separate experiment type, by separating the samples that contain NH2 in their name ##
pattern = r'^NH2.*'
data_not_nh2, data_nh2, target_not_nh2, target_nh2 = separateExperimentType(pattern, normalizedData, subsampledTarget)

In [ ]:
filename = "nonOriented"
mymst(data_not_nh2, target_not_nh2, colorDict, subFraction, filename)

In [ ]:
filename = "oriented"
mymst(data_nh2, target_nh2, colorDict, subFraction, filename)

### Silhouette

In [ ]:
# I re-write the labels so they are in the order that I want to plot:
uniqueLabelsSil = [
'NH2_pG1pM0',
'NH2_pG3pM1',
'NH2_pG1pM1',
'NH2_pG1pM3',
'NH2_pG0pM1',
'500_high',
'500_medium',
'500_low',
'300_high',
'300_medium',
'300_low',
'200_high',
'200_medium',
'200_low',
'100_high',
'100_medium']

In [ ]:
silhouetteCoefficient(scaledX, targetMass, uniqueLabelsSil, colorDict, labelHeader, pairwise=False, figsize=(17,7)) # Calculate the silhouette coefficient --> quality measure of clustering

## Compare
Pairwise comparison of samples

### Import and clean date dataset

In [ ]:
## Import the dataset separated by experiment date
pathDate = 'C:/Users/20210014/OneDrive - TU Eindhoven/testingData/_nanoparticleAtlas/npAtlas_reproByDay' # Pay attention to the direction of the slash "/"
dataDate = importData(pathDate)
dataDate

In [ ]:
## Clean Date Dataset
unfilteredDatasetDate = npAtlasDataCleaning(dataDate)
unfilteredDatasetDate

In [ ]:
option = 'fittingQuality'
filteredDatasetDate = filterDataset(option, unfilteredDatasetDate)
filteredDatasetDate # Show dataset

In [ ]:
list(filteredDatasetDate['sample'].unique())

### Initialize pipeline

In [ ]:
#Create color dictionary for the people reproducibility study
reprodColorDict = {'200_medium_VG':'#8ED081', '200_medium_MT':'#C09BEE', 
                    '300_medium_VG':'#8ED081', '300_medium_MT':'#C09BEE'}       #   E0B0D5   (PINK)  7BE0AD (GREEN)

In [ ]:
#Create color dictionary for the day reproducibility study
dateColorDict = {'300_medium_210616': '#ffd449',
                 '300_medium_210617': '#f9a010',
                 '300_medium_210630': '#a8d5e2',
                 '300_medium_211221': '#548c2f',
                 '300_medium_220103': '#104911'}

In [ ]:
# Initialize variables
renameHeaders = ['d', 'AR', 'Ab', 'Fab','Fc']
samples = ['300_medium_210616', '300_medium_210617', '300_medium_210630', '300_medium_211221', '300_medium_220103'] #Select samples to compare: '300_medium', '100_medium' | '300_medium_MT', '300_medium_VG' | '200_medium_MT', '200_medium_VG' | '300_medium_210616', '300_medium_210617', '300_medium_210630', '300_medium_211221', '300_medium_220103' | 'NH2_pG1pM0', 'NH2_pG0pM1'
color = dateColorDict # Select dictionary to use: colorDict | reprodColorDict | dateColorDict
pairwiseDataset = filteredDatasetDate #Select dataset to fetch the data from: filteredDataset | filteredReprod_peopleDataset | filteredDatasetDate
compare = True # Flag to plot metrics pairwise

In [ ]:
plotLegend(color) #Plot legend separately

In [ ]:
## Create pairwise dataset
# dataComp = pairwiseDataset[(pairwiseDataset[labelHeader] == samples[0]) | (pairwiseDataset[labelHeader] == samples[1])] # Extract values from defined pairwise samples

## Use this line for the timeline dataset (the full dataset is used here)
dataComp = pairwiseDataset

In [ ]:
list(dataComp[labelHeader].unique())

### Add control sample to comparison plot

In [ ]:
filteredDataset[filteredDataset[labelHeader] == '500_low']

In [ ]:
## Create pairwise dataset + ADD EXTRA SAMPLE FOR CONTROL
dataComp = pairwiseDataset[(pairwiseDataset[labelHeader] == samples[0]) | (pairwiseDataset[labelHeader] == samples[1])] # Extract values from defined pairwise samples
dataComp = pd.concat([dataComp, filteredDataset[(filteredDataset[labelHeader] == '500_low')]], ignore_index=True)
dataComp

In [ ]:
list(dataComp[labelHeader].unique())

In [ ]:
color = {'200_medium_VG':'#8ED081', '200_medium_MT':'#C09BEE', 
         '500_low':'#E3CAFB'}       #   E0B0D5   (PINK)  7BE0AD (GREEN)

### Metrics

In [ ]:
###
plt.subplots() # plot emtpy figure to set the style (idk why I get this bug)
sns.set_theme(style="ticks")
plt.close() # close empty figure
###

In [ ]:
## recalculate metrics for new labels
sPairwise, pdiPairwise, mcvPairwise, meanPairwise = metrics(dataComp, labelHeader)
smallerPairwise = reduceFeatures(dataComp)

In [ ]:
## Plot matrix with equal axis
metricsPlot(smallerPairwise, renameHeaders, mcvPairwise, samples, compare, labelHeader)

### UMAP

In [ ]:
###
plt.subplots() # plot emtpy figure to set the style (idk why I get this bug)
sns.set_theme(style="darkgrid")
plt.close() # close empty figure
###

In [ ]:
## Re-plot u-map
# 1. init dataset
minmax = False
pairwiseScaledX, _, pairwiseTargetMass, pairwiseUniqueLabels, _, pairwiseDataMass = preprocess4clustering(dataComp, labelHeader, meanPairwise, minmax) # Re-scale the pairwise data

In [ ]:
# 2. re-calculate
n_neighbors = 15
samplePlots = False # Plot each sample separately
reducer, embedding_df = myumap(pairwiseScaledX, pairwiseTargetMass, n_neighbors, pairwiseUniqueLabels, color, samplePlots) 

### Silhouette

In [ ]:
features = dataComp.columns.values # Get the features names (based on headers)
features = np.delete(features, -1) # Delete the last element (column 'sample') NOTE: THIS IS BECAUSE IN MY DATASETS, THE LABEL COLUMN IS ALWAYS THE LAST ONE, CHANGE ACCORDINGLY
dataSil = dataComp.loc[:, features].values # Separating out the features
targetSil = dataComp.loc[:, labelHeader].values
scaledSil = StandardScaler().fit_transform(dataSil) # Standardizing the features (z-score)

In [ ]:
silhouetteCoefficient(scaledSil, targetSil, samples, color, labelHeader, pairwise=True, figsize=(5,7)) # Calculate the silhouette coefficient --> quality measure of clustering

In [ ]:
## Use this cell for the timeline dataset
silhouetteCoefficient(dataSil, targetSil, samples, color, labelHeader, pairwise=False, figsize=(5,7)) # Calculate the silhouette coefficient --> quality measure of clustering

In [ ]:
feature = 'Diameter'
fig = plt.figure(figsize=(5,10))
sns.boxplot(data=filteredDatasetDate, x=labelHeader, y=feature, palette=color) # channel 2 target count
plt.ylabel('')
plt.xlabel('Diameter', fontsize=18)
Path('BOXPLOT').mkdir(parents=True, exist_ok=True)
fig.savefig("BOXPLOT/diameter.svg", facecolor=(1,1,1,0), dpi=72, bbox_inches='tight')
fig.savefig("BOXPLOT/diameter.png", facecolor=(1,1,1,0), dpi=600, bbox_inches='tight')

In [ ]:
# Histogram plot of the diameter values
feature = 'Diameter'
fig = plt.figure(figsize=(5,10))
sns.histplot(data=filteredDatasetDate, x=feature, hue=labelHeader, palette=color, kde=True)


### Statistical test (Kruskal-Wallis)

In [ ]:
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def analyze_particle_sizes_from_df(df, diameter_col='Diameter', group_col='sample'):
    """
    Perform Kruskal-Wallis H-test on nanoparticle size distributions from a DataFrame.
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing the measurements
    diameter_col (str): Name of the column containing diameter measurements
    group_col (str): Name of the column containing group labels
    
    Returns:
    dict: Dictionary containing test statistics and visualization
    """
    # Get unique groups
    groups = df[group_col].unique()
    
    # Split data into groups
    data_groups = [df[df[group_col] == group][diameter_col].values for group in groups]
    
    # Perform Kruskal-Wallis H-test
    h_statistic, p_value = stats.kruskal(*data_groups)
    
    # Create a summary of the results
    results = {
        'h_statistic': h_statistic,
        'p_value': p_value,
        'significant': p_value < 0.05
    }
    
    # Create visualization
    plt.figure(figsize=(12, 5))
    
    # Create box plot
    plt.subplot(1, 2, 1)
    sns.boxplot(data=df, x=group_col, y=diameter_col)
    plt.title('Size Distribution by Group')
    plt.ylabel('Diameter (nm)')
    plt.xticks(rotation=45)
    
    # Create violin plot
    plt.subplot(1, 2, 2)
    sns.violinplot(data=df, x=group_col, y=diameter_col)
    plt.title('Size Distribution Density by Group')
    plt.ylabel('Diameter (nm)')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    
    return results

# Example usage with your DataFrame:
results = analyze_particle_sizes_from_df(filteredDatasetDate)

print(f"Kruskal-Wallis H-statistic: {results['h_statistic']:.4f}")
print(f"p-value: {results['p_value']:.4f}")
print(f"Statistically significant difference: {results['significant']}")

# Optional: Display summary statistics for each group
summary_stats = filteredDatasetDate.groupby('sample')['Diameter'].describe()
print("\nSummary Statistics:")
print(summary_stats)

In [ ]:
# Total sample size
total_n = len(filteredDatasetDate)

# Sample size per group
group_sizes = filteredDatasetDate.groupby('sample').size()

print("\nSample Sizes:")
print(group_sizes)
print(f"\nTotal Sample Size: {total_n}")

In [ ]:
df = filteredDatasetDate
diameter_col='Diameter'
group_col='sample'

# Get unique groups
groups = df[group_col].unique()

# Split data into groups
data_groups = [df[df[group_col] == group][diameter_col].values for group in groups]

data_groups

In [ ]:
from scipy import stats
import scikit_posthocs as sp

dunn = sp.posthoc_dunn(data_groups, p_adjust='bonferroni')

In [ ]:

dunn < 0.00005 # 1 *

### Interactive UMAP for the timeline dataset

In [ ]:
mapper = reducer.fit(pairwiseScaledX)

In [ ]:
hover_data = pd.DataFrame({'index':np.arange(len(pairwiseDataMass)),
                           'Diameter':pairwiseDataMass[:,0],
                           'Aspect ratio': pairwiseDataMass[:,1],
                           'Total Ab': pairwiseDataMass[:,2],
                           'Fab':pairwiseDataMass[:,13],
                           'Fc': pairwiseDataMass[:,22],
                           'Sample': pairwiseTargetMass})

In [ ]:
import umap.plot as umap_plot
umap_plot.output_notebook()

In [ ]:
p = umap_plot.interactive(mapper, labels=pairwiseTargetMass, hover_data=hover_data, point_size=2, background='#EAEAF2')
umap_plot.show(p)

### Color per feature for the timeline dataset

In [ ]:
featureToColor = ['Diameter', 'Aspect ratio', 'Cluster localizations', 'channel 1 target count', 'channel 2 target count', 'channel 1 true mean dark time', 'channel 2 true mean dark time', 'channel 1 mean bright time', 'channel 2 mean bright time']
featureColoredUmap(filteredDatasetDate, featureToColor, embedding_df, 'umap_1', 'umap_2')

# TODO: Plot the aspect ratio separately to control the legend, since the outlier values are biasing the colors (or clean the dataset from those outliers or use a continuous coloring (colorbar))